In [1]:
using RCall, Distributions, Gadfly, MultivariateStats
include("../multiallelegibbs.jl");
include("../eigenanlaysis.jl");
include("../singleallelesampler.jl");

In [2]:
rcopy("source('../pedR2jl.R')");

In [3]:
geno = rcopy("X");
ped = rcopy("numped");
rcopy("rm(list=ls())");
rcopy("gc()");

In [8]:
n,m = size(geno);
iter = 10000;
thin = 20;
nsamp = length(thin:thin:iter);

In [9]:
@time pedfit = pedigree_genogibbs(geno,ped,iter,thin);

elapsed time: 525.56644283 seconds (160393079528 bytes allocated, 33.32% gc time)


In [7]:
print("Genotype array: ",size(pedfit[4]),"\n")
print("MAF array: ",size(pedfit[2]),"\n")
print("Error rate array: ",size(pedfit[3]))

Genotype array: (2,944,70,100)
MAF array: (70,100)
Error rate array: (100,)

In [8]:
path = "/home/seth/analysis/ppc4ped/";
writecsv(string(path,"eigenanalysis_geno.csv"),pedfit[4][:]);
writecsv(string(path,"eigenanalysis_maf.csv"),pedfit[2]);
writecsv(string(path,"eigenanalysis_errate.csv"),pedfit[3]);

If necessary, load samples back up rather than rerunning the sampler.

In [38]:
path = "/home/seth/analysis/ppc4ped/";
mafs = readcsv(string(path,"eigenanalysis_maf.csv"),Float64);
gs = readcsv(string(path,"eigenanalysis_geno.csv"),Int8);
errs = readcsv(string(path,"eigenanalysis_errate.csv"));

gs = reshape(gs,(2,n,m,nsamp));
gs = reshape(sum(gs,1),(n,m,nsamp))
pedfit = (gs,mafs,errs);

In [67]:
genotyped = (sum(geno.!=-1,2).>1)[:];
m = size(geno,2);
ng = countnz(genotyped);
nsim = 10;

nullgeno = Array(Int8,(ng,m,nsim));
for t in 1:nsim
    for j in 1:m
        nullgeno[:,j,t] = rand(Binomial(2,pedfit[2][j,t]),ng);
    end
end

ppgeno = Array(Int8,(n,m,nsim));
for t in 1:nsim
    for j in 1:m
        ppgeno[:,j,t] = pedigree_genosim(ped,pedfit[2][j,t])[1];
    end
end
ppgeno = ppgeno[genotyped,:,:];

In [70]:
peig = pedigree_eigenanalysis(pedfit[1][genotyped,:,:]);
ppeig = pedigree_eigenanalysis(ppgeno);
nulleig = pedigree_eigenanalysis(nullgeno);